In [22]:
import pandas as pd
from itertools import cycle
import gpflow
import numpy as np
from gpflow.utilities import print_summary, positive
from gpflow.ci_utils import ci_niter
from gpflow.optimizers import NaturalGradient
from gpflow import set_trainable
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import namedtuple
import statsmodels.api as sm
import dill as pickle

sns.set(style="white")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

gpflow.config.set_default_float(np.float64)
gpflow.config.set_default_summary_fmt("notebook")

<ipython-input-22-06da3225ef62>:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [23]:
final_data = pd.read_pickle("tmp/final_data.pkl")
results_states = pickle.load(open('tmp/trained_models_states.pkl', 'rb'))
linear_filters_states = pickle.load(open('tmp/linear_filters_states.pkl', 'rb'))
rhos = pickle.load(open("tmp/rhos.pkl", "rb"))

In [24]:
ORDER = len(list(linear_filters_states.values())[0][0])
N = len(linear_filters_states)
T2 = int(rhos.shape[0] / N)

## Marginal Kernels

In [25]:
def prepare_data_for_state(data):
    data = data.copy()
    data.reset_index(inplace=True)
    t = data[["Updated"]]
    y = data[["Updated", "ConfirmedChange"]]
    X = pd.DataFrame({
        "Updated": data["Updated"],
        "state": data["state"].apply(lambda v: list(results_states.keys()).index(v)),
        "index": data["index"],
        "S": data["S"],
        "I": data["I"],
        "HOME": data["HOME"],
        "WORK": data["WORK"],
        "RECREATIONAL": data["RECREATIONAL"],
    }, index=data.index)[[
        "Updated",
        "state",
        "index",
        "S",
        "I",
        "HOME", 
        "WORK", 
        "RECREATIONAL"
    ]]
    return X, y

In [26]:
times = None
for s in final_data.index.unique():
    s_times = set(final_data.loc[s, 'Updated'].unique())
    if times is None:
        times = s_times
    else:
        times = times.intersection(s_times)

In [136]:
ks = np.zeros((T2*N, T2))

final_data_ = final_data[final_data['Updated'].isin(times)]

for i, (s, (m, _r)) in enumerate(results_states.items()):
    data = final_data_.loc[s]
    X, _y = prepare_data_for_state(data)
    Xnew = X.drop(columns=["Updated", "index"]).values[ORDER-1:]
    
    kernel = m._model.kernel
    inducing_variable = m._model.inducing_variable
    f = m._model.q_mu
    
    q_sqrt = m._model.q_sqrt
    full_cov = True
    
    Kmm = kernel(inducing_variable.Z)
    Kmm += 1e-6 * tf.eye(inducing_variable.num_inducing, dtype=Kmm.dtype)

    Kmn = kernel(inducing_variable.Z, Xnew)
    
    Knn = kernel(Xnew, full_cov=full_cov)
    
    Lm = tf.linalg.cholesky(Kmm)
    num_func = tf.shape(f)[-1]  # R
    N = tf.shape(Kmn)[-1]
    
    K = tf.rank(Kmn)
    perm = tf.concat(
        [
            tf.reshape(tf.range(1, K - 1), [K - 2]),  # leading dims (...)
            tf.reshape(0, [1]),  # [M]
            tf.reshape(K - 1, [1]),
        ],
        0,
    )  # [N]
    Kmn = tf.transpose(Kmn, perm)  # [..., M, N]

    leading_dims = tf.shape(Kmn)[:-2]

    # Compute the projection matrix A
    Lm = tf.broadcast_to(Lm, tf.concat([leading_dims, tf.shape(Lm)], 0))  # [..., M, M]
    A = tf.linalg.triangular_solve(Lm, Kmn, lower=True)  # [..., M, N]
    
    if full_cov:
        fvar = Knn - tf.linalg.matmul(A, A, transpose_a=True)  # [..., N, N]
        cov_shape = tf.concat([leading_dims, [num_func, N, N]], 0)
        fvar = tf.broadcast_to(tf.expand_dims(fvar, -3), cov_shape)  # [..., R, N, N]
    else:
        fvar = Knn - tf.reduce_sum(tf.square(A), -2)  # [..., N]
        cov_shape = tf.concat([leading_dims, [num_func, N]], 0)  # [..., R, N]
        fvar = tf.broadcast_to(tf.expand_dims(fvar, -2), cov_shape)  # [..., R, N]
        
    q_sqrt_dims = q_sqrt.shape.ndims
    if q_sqrt_dims == 2:
        LTA = A * tf.expand_dims(tf.transpose(q_sqrt), 2)  # [R, M, N]
    elif q_sqrt_dims == 3:
        L = tf.linalg.band_part(q_sqrt, -1, 0)  # force lower triangle # [R, M, M]
        L_shape = tf.shape(L)
        L = tf.broadcast_to(L, tf.concat([leading_dims, L_shape], 0))

        shape = tf.concat([leading_dims, [num_func, M, N]], axis=0)
        A_tiled = tf.broadcast_to(tf.expand_dims(A, -3), shape)
        LTA = tf.linalg.matmul(L, A_tiled, transpose_a=True)  # [R, M, N]
    else:  # pragma: no cover
        raise ValueError("Bad dimension for q_sqrt: %s" % str(q_sqrt.shape.ndims))

    if full_cov:
        fvar = fvar + tf.linalg.matmul(LTA, LTA, transpose_a=True)  # [R, N, N]
    else:
        fvar = fvar + tf.reduce_sum(tf.square(LTA), -2)  # [R, N]

    if not full_cov:
        fvar = tf.linalg.adjoint(fvar)  # [N, R]
    
    break

fvar
B = fvar.numpy()[0]

np.all(np.linalg.eigvals(B) > 0)

False

In [1002]:
M = ks + rhos

In [1003]:
M.shape

(2516, 2516)

In [1004]:
np.linalg.matrix_rank(M)

880

In [1005]:
np.linalg.det(M)

0.0

In [1006]:
e_val, e_vec = np.linalg.eig(M)
np.max(e_val) / np.min(e_val)

(-546082.4507611456-0j)

In [863]:
M_reshaped = np.zeros((T2, N, T2, N))

for i in range(N):
    for j in range(N):
        v = M[i*T2:(i+1)*T2, j*T2:(j+1)*T2]
        M_reshaped[:, i, :, j] = v

In [864]:
M_reshaped.shape

(148, 17, 148, 17)